TODO
* test with top 3 tfidf
* test na prep final results


* Code to generate our topics for Anserini (extend UDel, and alternative to UDel)
* Test task_short_title
* Test task_NER_long
* Make sure we replicate https://github.com/castorini/anserini/blob/master/src/main/python/trec-covid/generate_query_udel.py
* Celebrate

In [6]:
#quick query term test
from pyserini.search import SimpleSearcher
#from pyserini.index import pyutils
#from pyserini.analysis import pyanalysis
#from pyserini.search import pysearch
#from pyserini.search import pyquerybuilder
#from pyserini.pyclass import autoclass
#from pyserini.analysis.pyanalysis import get_lucene_analyzer

index_loc = '/home/tmschoegje/Desktop/anserini/indexes/lucene-index-cord19-full-text-2020-05-19/'
searcher = SimpleSearcher(index_loc)
#index_utils = pyutils.IndexReaderUtils(index_loc)


hits = searcher.search('corona origin')

# Print the first 10 hits:
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')
    
    
hits = searcher.search('corona origin origin origin')
print()
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')


 1 rflu6axg        4.73290
 2 6f3up1h0        4.67620
 3 u13oge9m        4.58750
 4 z6f56n7h        4.53720
 5 ipk43u69        4.49140
 6 js9ek5me        4.42320
 7 r6wwpfcy        4.41610
 8 gzxu0r08        4.40190
 9 r68l7g2e        4.39970
10 pyeb86on        4.38390

 1 rflu6axg        7.56320
 2 u13oge9m        7.46630
 3 6f3up1h0        7.30750
 4 r6wwpfcy        7.07580
 5 r68l7g2e        7.07000
 6 pyeb86on        7.00350
 7 rpf02tc0        6.98700
 8 pe39ov47        6.97700
 9 yrz8s11w        6.95560
10 y4zpl1ji        6.85060


In [32]:
import scispacy
import spacy
import xml.etree.ElementTree as Et
import nltk
import pandas as pd
import numpy as np

task_1_short = "transmission incubation environment"
task_2_short = "risk factors"
task_3_short = "genetics origin evolution"
task_4_short = "vaccines therapeutics"
task_5_short = "medical care"
task_6_short = "effectiveness non pharmaceutical interventions"
task_7_short = "diagnostics surveillance"
task_8_short = "geographic spread"
task_9_short = "ethical social considerations"
task_10_short = "information sharing collaboration"

task_1 = "What is known about transmission, incubation, and environmental stability? What do we know about natural history, transmission, and diagnostics for the virus? What have we learned about infection prevention and control? Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery. Prevalence of asymptomatic shedding and transmission (e.g., particularly children). Seasonality of transmission. Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding). Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood). Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic). Natural history of the virus and shedding of it from an infected person. Implementation of diagnostics and products to improve clinical processes. Disease models, including animal models for infection, disease and transmission. Tools and studies to monitor phenotypic change and potential adaptation of the virus. Immune response and immunity. Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings. Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings. Role of the environment in transmission"
task_2 = "What do we know about COVID-19 risk factors? What have we learned from epidemiological studies? Data on potential risks factors. Smoking, pre-existing pulmonary disease. Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-morbidities. Neonates and pregnant women. Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences. Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors. Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups. Susceptibility of populations. Public health mitigation measures that could be effective for control."
task_3 = "What do we know about virus genetics, origin, and evolution? What do we know about the virus origin and management measures at the human-animal interface? Real-time tracking of whole genomes and a mechanism for coordinating the rapid dissemination of that information to inform the development of diagnostics and therapeutics and to track variations of the virus over time. Access to geographic and temporal diverse sample sets to understand geographic distribution and genomic differences, and determine whether there is more than one strain in circulation. Multi-lateral agreements such as the Nagoya Protocol could be leveraged. Evidence that livestock could be infected (e.g., field surveillance, genetic sequencing, receptor binding) and serve as a reservoir after the epidemic appears to be over. Evidence of whether farmers are infected, and whether farmers could have played a role in the origin. Surveillance of mixed wildlife- livestock farms for SARS-CoV-2 and other coronaviruses in Southeast Asia. Experimental infections to test host range for this pathogen. Animal host(s) and any evidence of continued spill-over to humans. Socioeconomic and behavioral risk factors for this spill-over. Sustainable risk reduction strategies"
task_4 = "What do we know about vaccines and therapeutics? What has been published concerning research and development and evaluation efforts of vaccines and therapeutics? Effectiveness of drugs being developed and tried to treat COVID-19 patients. Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocycline that that may exert effects on viral replication. Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients. Exploration of use of best animal models and their predictive value for a human vaccine. Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents. Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up. This could include identifying approaches for expanding production capacity to ensure equitable and timely distribution to populations in need. Efforts targeted at a universal coronavirus vaccine. Efforts to develop animal models and standardize challenge studies. Efforts to develop prophylaxis clinical studies and prioritize in healthcare workers. Approaches to evaluate risk for enhanced disease after vaccination. Assays to evaluate vaccine immune response and process development for vaccines, alongside suitable animal models in conjunction with therapeutics."
task_5 = "What has been published about medical care? What has been published concerning surge capacity and nursing homes? What has been published concerning efforts to inform allocation of scarce resources? What do we know about personal protective equipment? What has been published concerning alternative methods to advise on disease management? What has been published concerning processes of care? What do we know about the clinical characterization and management of the virus? Resources to support skilled nursing facilities and long term care facilities. Mobilization of surge medical staff to address shortages in overwhelmed communities. Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies. Extracorporeal membrane oxygenation (ECMO) outcomes data of COVID-19 patients. Outcomes data for COVID-19 after mechanical ventilation adjusted for age. Application of regulatory standards (e.g., EUA, CLIA) and ability to adapt care to crisis standards of care level. Approaches for encouraging and facilitating the production of elastomeric respirators, which can save thousands of N95 masks. Best telemedicine practices, barriers and facilitators, and specific actions to remove/expand them within and across state boundaries. Guidance on the simple things people can do at home to take care of sick people and manage disease. Oral medications that might potentially work. Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually. Best practices and critical challenges and innovative solutions and technologies in hospital flow and organization, workforce protection, workforce allocation, community-based support resources, payment, and supply chain management to enhance capacity, efficiency, and outcomes. Efforts to define the natural history of disease to inform clinical care, public health interventions, infection prevention control, transmission, and clinical trials. Efforts to develop a core clinical outcome set to maximize usability of data across a range of trials. Efforts to determine adjunctive and supportive interventions that can improve the clinical outcomes of infected patients (e.g. steroids, high flow oxygen). "
task_6 = "What do we know about the effectiveness of non-pharmaceutical interventions? What is known about equity and barriers to compliance for non-pharmaceutical interventions? Guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases. Rapid design and execution of experiments to examine and compare NPIs currently being implemented. DHS Centers for Excellence could potentially be leveraged to conduct these experiments. Rapid assessment of the likely efficacy of school closures, travel bans, bans on mass gatherings of various sizes, and other social distancing approaches. Methods to control the spread in communities, barriers to compliance and how these vary among different populations. Models of potential interventions to predict costs and benefits that take account of such factors as race, income, disability, age, geographic location, immigration status, housing status, employment status, and health insurance status. Policy changes necessary to enable the compliance of individuals with limited resources and the underserved with NPIs. Research on why people fail to comply with public health advice, even if they want to do so (e.g., social or financial costs may be too high). Research on the economic impact of this or any pandemic. This would include identifying policy and programmatic alternatives that lessen/mitigate risks to critical government services, food distribution and supplies, access to critical household supplies, and access to health diagnoses, treatment, and needed care, regardless of ability to pay."
task_7 = "What do we know about diagnostics and surveillance? What has been published concerning systematic, holistic approach to diagnostics (from the public health surveillance perspective to being able to predict clinical outcomes)? How widespread current exposure is to be able to make immediate policy recommendations on mitigation measures. Denominators for testing and a mechanism for rapidly sharing that information, including demographics, to the extent possible. Sampling methods to determine asymptomatic disease (e.g., use of serosurveys (such as convalescent samples) and early detection of disease (e.g., use of screening of neutralizing antibodies such as ELISAs). Efforts to increase capacity on existing diagnostic platforms and tap into existing surveillance platforms. Recruitment, support, and coordination of local expertise and capacity (public, private—commercial, and non-profit, including academic), including legal, ethical, communications, and operational issues. National guidance and guidelines about best practices to states (e.g., how states might leverage universities and private laboratories for testing purposes, communications to public health officials and the public). Development of a point-of-care test (like a rapid influenza test) and rapid bed-side tests, recognizing the tradeoffs between speed, accessibility, and accuracy. Rapid design and execution of targeted surveillance experiments calling for all potential testers using PCR in a defined area to start testing and report to a specific entity. These experiments could aid in collecting longitudinal samples, which are critical to understanding the impact of ad hoc local interventions (which also need to be recorded). Separation of assay development issues from instruments, and the role of the private sector to help quickly migrate assays onto those devices. Efforts to track the evolution of the virus (i.e., genetic drift or mutations) and avoid locking into specific reagents and surveillance/detection schemes. Latency issues and when there is sufficient viral load to detect the pathogen, and understanding of what is needed in terms of biological and environmental sampling. Use of diagnostics such as host response markers (e.g., cytokines) to detect early disease or predict severe disease progression, which would be important to understanding best clinical practice and efficacy of therapeutic interventions. Policies and protocols for screening and testing. Policies to mitigate the effects on supplies associated with mass testing, including swabs and reagents. Technology roadmap for diagnostics. Barriers to developing and scaling up new diagnostic tests (e.g., market forces), how future coalition and accelerator models (e.g., Coalition for Epidemic Preparedness Innovations) could provide critical funding for diagnostics, and opportunities for a streamlined regulatory environment. New platforms and technology (e.g., CRISPR) to improve response times and employ more holistic approaches to COVID-19 and future diseases. Coupling genomics and diagnostic testing on a large scale. Enhance capabilities for rapid sequencing and bioinformatics to target regions of the genome that will allow specificity for a particular variant. Enhance capacity (people, technology, data) for sequencing with advanced analytics for unknown pathogens, and explore capabilities for distinguishing naturally-occurring pathogens from intentional. One Health surveillance of humans and potential sources of future spillover or ongoing exposure for this organism and future pathogens, including both evolutionary hosts (e.g., bats) and transmission hosts (e.g., heavily trafficked and farmed wildlife and domestic food and companion species), inclusive of environmental, demographic, and occupational risk factors."
task_8 = "At the time of writing, COVID-19 has spread to at least 114 countries. With viral flu, there are often geographic variations in how the disease will spread and if there are different variations of the virus in different areas. We’d like to explore what the literature and data say about this through this Task. Are there geographic variations in the rate of COVID-19 spread? Are there geographic variations in the mortality rate of COVID-19? Is there any evidence to suggest geographic based virus mutations?"
task_9 = "What has been published concerning ethical considerations for research? What has been published concerning social sciences at the outbreak response? Efforts to articulate and translate existing ethical principles and standards to salient issues in COVID-2019. Efforts to embed ethics across all thematic areas, engage with novel ethical issues that arise and coordinate to minimize duplication of oversight. Efforts to support sustained education, access, and capacity building in the area of ethics. Efforts to establish a team at WHO that will be integrated within multidisciplinary research and operational platforms and that will connect with existing and expanded global networks of social sciences. Efforts to develop qualitative assessment frameworks to systematically collect information related to local barriers and enablers for the uptake and adherence to public health measures for prevention and control. This includes the rapid identification of the secondary impacts of these measures. (e.g. use of surgical masks, modification of health seeking behaviors for SRH, school closures). Efforts to identify how the burden of responding to the outbreak and implementing public health measures affects the physical and psychological health of those providing care for Covid-19 patients and identify the immediate needs that must be addressed. Efforts to identify the underlying drivers of fear, anxiety and stigma that fuel misinformation and rumor, particularly through social media."
task_10 = "What has been published about information sharing and inter-sectoral collaboration? What has been published about data standards and nomenclature? What has been published about governmental public health? What do we know about risk communication? What has been published about communicating with high-risk populations? What has been published to clarify community measures? What has been published about equity considerations and problems of inequity? Methods for coordinating data-gathering with standardized nomenclature. Sharing response information among planners, providers, and others. Understanding and mitigating barriers to information-sharing. How to recruit, support, and coordinate local (non-Federal) expertise and capacity relevant to public health emergency response (public, private, commercial and non-profit, including academic). Integration of federal/state/local public health surveillance systems. Value of investments in baseline public health response infrastructure preparedness. Modes of communicating with target high-risk populations (elderly, health care workers). Risk communication and guidelines that are easy to understand and follow (include targeting at risk populations’ families too). Communication that indicates potential risk of disease to all population groups. Misunderstanding around containment and mitigation. Action plan to mitigate gaps and problems of inequity in the Nation’s public health capability, capacity, and funding to ensure all citizens in need are supported and can access information, surveillance, and treatment. Measures to reach marginalized and disadvantaged populations. Data systems and research priorities and agendas incorporate attention to the needs and circumstances of disadvantaged populations and underrepresented minorities. Mitigating threats to incarcerated people from COVID-19, assuring access to information, prevention, diagnosis, and treatment. Understanding coverage policies (barriers and opportunities) related to testing, treatment, and care."

list_of_tasks = [task_1, task_2, task_3, task_4, task_5, task_6, task_7, task_8, task_9, task_10]
list_of_tasks_short = [task_1_short, task_2_short, task_3_short, task_4_short, task_5_short, task_6_short, task_7_short, task_8_short, task_9_short, task_10_short]

#classification with confidence
rnd3classes = [2, 0, 3, 0, 3, 7, 7, 7, 6, 5, 4, 5, 0, 0, 0, 0, 4, 5, 5, 1, 0, 1, 1, 1, 1, 7, 7, 3, 3, 3, 2, 2, 3, 3, 9, 2, 2, 3, 3, 2]
rnd3confidence = [1, 1, 1, 0.5, 0.5, 1, 1, 0.5, 0.5, 1, 0, 0.5, 1, 0, 1, 1, 0.75, 0.25, 0.25, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 1, 0.5, 1, 1, 1, 0, 1, 1, 0, 0, 0.5, 1, 0, 0, 1]


stopwords = {
        'a', 'about', 'above', 'according', 'across', 'after', 'afterwards', 'again', 
        'against', 'albeit', 'all', 'almost', 'alone', 'along', 'already', 'also', 
        'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 
        'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 
        'apart', 'are', 'around', 'as', 'at', 'av', 'be', 'became', 'because', 
        'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 
        'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'but', 
        'by', 'can', 'cannot', 'canst', 'certain', 'cf', 'choose', 'contrariwise', 
        'cos', 'could', 'cu', 'day', 'do', 'does', 'doesn', 't', 'doing', 'dost', 
        'doth', 'double', 'down', 'dual', 'during', 'each', 'either', 'else', 
        'elsewhere', 'enough', 'et', 'etc', 'even', 'ever', 'every', 'everybody', 
        'everyone', 'everything', 'everywhere', 'except', 'excepted', 'excepting', 
        'exception', 'exclude', 'excluding', 'exclusive', 'far', 'farther', 
        'farthest', 'few', 'ff', 'first', 'for', 'formerly', 'forth', 'forward', 
        'from', 'front', 'further', 'furthermore', 'furthest', 'get', 'go', 'had', 
        'halves', 'hardly', 'has', 'hast', 'hath', 'have', 'he', 'hence', 
        'henceforth', 'her', 'here', 'hereabouts', 'hereafter', 'hereby', 'herein', 
        'hereto', 'hereupon', 'hers', 'herself', 'him', 'himself', 'hindmost', 'his', 
        'hither', 'hitherto', 'how','however', 'howsoever', 'i', 'ie', 'if', 'in', 
        'inasmuch', 'inc', 'include', 'included', 'including', 'indeed', 'indoors', 
        'inside', 'insomuch', 'instead', 'into', 'inward', 'inwards', 'is', 'it', 
        'its', 'itself', 'just', 'kg', 'kind', 'km', 'last', 'latter', 'latterly', 
        'less', 'lest', 'let', 'like', 'little', 'ltd', 'many', 'may', 'maybe', 'me', 
        'meantime', 'meanwhile', 'might', 'more','moreover', 'most', 'mostly', 'mr', 
        'mrs', 'ms', 'much', 'must', 'my', 'myself', 'namely', 'need', 'neither', 
        'never', 'nevertheless', 'next', 'no', 'nobody', 'none', 'nonetheless', 
        'noone', 'nope', 'nor', 'not', 'nothing', 'notwithstanding', 'now', 
        'nowadays', 'nowhere', 'of', 'off', 'often', 'ok', 'on', 'once', 'one', 
        'only', 'onto', 'or', 'other', 'others', 'otherwise', 'ought', 'our', 
        'ours', 'ourselves', 'out', 'outside', 'over', 'own', 'per', 'perhaps', 
        'plenty', 'provide', 'quite', 'rather', 'really', 'round', 'said', 'sake', 
        'same', 'sang', 'save', 'saw', 'see', 'seeing', 'seem', 'seemed', 'seeming', 
        'seems', 'seen', 'seldom', 'selves', 'sent', 'several', 'shalt', 'she', 
        'should', 'shown', 'sideways', 'since', 'slept', 'slew', 'slung', 'slunk', 
        'smote', 'so', 'some', 'somebody', 'somehow', 'someone', 'something', 
        'sometime', 'sometimes', 'somewhat', 'somewhere', 'spake', 'spat', 'spoke', 
        'spoken', 'sprang', 'sprung', 'stave', 'staves', 'still', 'such', 'supposing', 
        'than', 'that', 'the', 'thee', 'their', 'them', 'themselves', 'then', 'thence', 
        'thenceforth', 'there', 'thereabout', 'thereabouts', 'thereafter', 'thereby', 
        'therefore', 'therein', 'thereof', 'thereon', 'thereto', 'thereupon', 'these', 
        'they', 'this', 'those', 'thou', 'though', 'thrice', 'through', 'throughout', 
        'thru', 'thus', 'thy', 'thyself', 'till', 'to', 'together', 'too', 'toward', 
        'towards', 'ugh', 'unable', 'under', 'underneath', 'unless', 'unlike', 'until', 
        'up', 'upon', 'upward', 'upwards', 'us', 'use', 'used', 'using', 'very', 'via', 
        'vs', 'want', 'was', 'we', 'week', 'well', 'were', 'what', 'whatever', 
        'whatsoever', 'when', 'whence', 'whenever', 'whensoever', 'where', 'whereabouts', 
        'whereafter', 'whereas', 'whereat', 'whereby', 'wherefore', 'wherefrom', 
        'wherein', 'whereinto', 'whereof', 'whereon', 'wheresoever', 'whereto', 'whereunto', 
        'whereupon', 'wherever', 'wherewith', 'whether', 'whew', 'which', 'whichever', 
        'whichsoever', 'while', 'whilst', 'whither', 'who', 'whoa', 'whoever', 'whole', 
        'whom', 'whomever', 'whomsoever', 'whose', 'whosoever', 'why', 'will', 'wilt', 
        'with', 'within', 'without', 'worse', 'worst', 'would', 'wow', 'ye', 'year', 'yet', 
        'yippee', 'you', 'your', 'yours', 'yourself', 'yourselves'
    }

In [18]:
#read in udel topics, create task_short
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3.xml"
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-udel.xml"
write_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-ruir.short.xml"
topics_n = 40

tree = Et.parse(topics_path)
root = tree.getroot()

for i in range(topics_n):
    root[i][0].text += " " + list_of_tasks_short[rnd3classes[i]]

#topics = [root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]] for i in range(topics_n)]
#print(topics)

#topics = [t + " " + list_of_tasks_short[rnd3classes[i]] for i,t in enumerate(topics)]
#print(topics)

tree.write(open(write_path, 'wb'))


In [19]:
#read in udel topics, create task_only
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3.xml"
write_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-ruir.task.short.xml"
topics_n = 40

tree = Et.parse(topics_path)
root = tree.getroot()

for i in range(topics_n):
    root[i][0].text += " " + list_of_tasks_short[rnd3classes[i]]

#topics = [root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]] for i in range(topics_n)]
#print(topics)

#topics = [t + " " + list_of_tasks_short[rnd3classes[i]] for i,t in enumerate(topics)]
#print(topics)

tree.write(open(write_path, 'wb'))


In [29]:
#scispacy NER on tasks to find terms
#TODO 

task_ner = []
nlp = spacy.load("en_core_sci_sm")
for task in list_of_tasks:
    doc = nlp(task)
    ents = ""
    for e in doc.ents:
        ents += " " + str(e)
    task_ner.append(ents)

#create task_long
#read in udel topics, create task_short
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3.xml"
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-udel.xml"
write_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-ruir.long.xml"
topics_n = 40

tree = Et.parse(topics_path)
root = tree.getroot()

for i in range(topics_n):
    root[i][0].text += task_ner[rnd3classes[i]]

#topics = [root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]] for i in range(topics_n)]
#print(topics)

#topics = [t + " " + list_of_tasks_short[rnd3classes[i]] for i,t in enumerate(topics)]
#print(topics)

tree.write(open(write_path, 'wb'))

In [21]:
#load metadata.csv
def prepTREC(fname):
    #get valid TREC ids for this round
    TRECids = []
    f = open(fname)
    for line in f.readlines():
        if line[-1] == '\n':
            line = line[:-1]
        TRECids.append(line)
    f.close()
    
    metadata = pd.read_csv("/home/tmschoegje/Desktop/caos-19/metadata.csv")
    #now we filter all TREC ids we don't need
    #print(np.array(metadata).shape)
    #print(np.array(TRECids).shape)
    metadata = metadata[metadata.cord_uid.isin(TRECids)]
    #there's 33 duplicates (same paper from multiple sources)
    metadata.drop_duplicates(subset='cord_uid', keep='first', inplace=True)
    
    return metadata

In [26]:
#let's find the tfidf task terms

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


qrels = []         #contains topic, cord_uid, qrel, assessround qrels
doc_sets = []      #contains qrels per task
journal_sets = []  #contains individual fulltext abstracts

for line in open('/home/tmschoegje/Desktop/caos-19/trecdata/qrels-covid_d3_j0.5-3.txt').readlines():
    vals = line.strip().split(" ")
    #if relevant, add
    if(float(vals[3]) > 0):
        #topic, cord_uid, qrel, assessround
        qrels.append([int(vals[0]), vals[2], float(vals[3]), float(vals[1])])
qrels = np.array(qrels, dtype="O")

sw = stopwords.words("english")
porter = PorterStemmer()

#finally, we also want to use the other abstracts for our corpus (so our tf idf scores get better)
corpus = []
metadata = prepTREC('/home/tmschoegje/Desktop/caos-19/trecdata/docids-rnd3.txt')
for row in np.array(metadata):
    if row[0] not in qrels[:,0]:
        corpus.append(str(row[8]))
        #corpus += str(row[8])
        
#Now start computing tf-idf
cv = CountVectorizer(max_df=0.85, min_df=0.05, stop_words = sw)
word_count_vector = cv.fit_transform(corpus)

tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

feature_names = cv.get_feature_names()


#helper functions for below
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
    
def get_topn(feature_names, sorted_vectors, topn=10):
    sorted_vectors = sorted_vectors[:topn]
    
    scores = []
    features = []
    
    for ind, score in sorted_vectors:
        scores.append(round(score, 3))
        features.append(feature_names[ind])
        
    results = {}
    for ind in range(len(features)):
        results[features[ind]] = scores[ind]
        
    return results

def get_tfidf(task_list, topn):
    task_keywords = []
    for task_id, task_text in enumerate(task_list):
        tfidf_vector = tfidf_transformer.transform(cv.transform([task_text]))
        #print(tfidf_vector)

        #sort tf idf vectors by scores
        sorted_vectors = sort_coo(tfidf_vector.tocoo())

        #get top keywords for task w/ score
        keywords = get_topn(feature_names, sorted_vectors, topn)
        task_keywords.append(keywords)
    return task_keywords

#    print(keywords)
#    task_keywords.append(keywords)

for task in list_of_tasks:
    print(get_tfidf(list_of_tasks, 3))

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


[{'transmission': 0.792, 'virus': 0.256, 'health': 0.232}, {'factors': 0.531, 'risk': 0.351, 'transmission': 0.279}, {'evidence': 0.436, 'could': 0.405, 'host': 0.303}, {'studies': 0.408, 'clinical': 0.359, 'development': 0.302}, {'care': 0.599, 'outcomes': 0.408, 'clinical': 0.296}, {'health': 0.493, 'research': 0.305, 'non': 0.288}, {'including': 0.379, 'public': 0.372, 'disease': 0.248}, {'spread': 0.51, 'covid': 0.423, '19': 0.392}, {'health': 0.436, 'measures': 0.433, 'identify': 0.426}, {'public': 0.486, 'risk': 0.414, 'information': 0.407}]
[{'transmission': 0.792, 'virus': 0.256, 'health': 0.232}, {'factors': 0.531, 'risk': 0.351, 'transmission': 0.279}, {'evidence': 0.436, 'could': 0.405, 'host': 0.303}, {'studies': 0.408, 'clinical': 0.359, 'development': 0.302}, {'care': 0.599, 'outcomes': 0.408, 'clinical': 0.296}, {'health': 0.493, 'research': 0.305, 'non': 0.288}, {'including': 0.379, 'public': 0.372, 'disease': 0.248}, {'spread': 0.51, 'covid': 0.423, '19': 0.392}, {'hea

In [52]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tmschoegje/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
limit = 5
print(get_tfidf(task_ner, limit))
tfidf = get_tfidf(task_ner, limit + 3)

#read in basic topics, create ruir.qq.tfidf and ruir.qdel.tfidf
#topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3.xml"
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-udel.xml"
write_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-ruir.qdel.tfidf.xml"
topics_n = 40

tree = Et.parse(topics_path)
root = tree.getroot()

for i in range(topics_n):
    terms = [term for term in tfidf[rnd3classes[i]]]
    
    numterms = 0
    newterms = []
    for t in terms:
        if str(t) not in stopwords and numterms < limit:
            newterms.append(str(t))
            numterms += 1
    root[i][0].text += " " + " ".join(newterms)
    print(newterms)
    
#    list_of_tasks_short[rnd3classes[i]]

#topics = [root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]] for i in range(topics_n)]
#print(topics)

#topics = [t + " " + list_of_tasks_short[rnd3classes[i]] for i,t in enumerate(topics)]
#print(topics)

tree.write(open(write_path, 'wb'))


[{'transmission': 0.821, 'virus': 0.265, 'health': 0.24, 'disease': 0.202, 'care': 0.185}, {'factors': 0.56, 'risk': 0.37, 'transmission': 0.294, 'virus': 0.285, 'infections': 0.253}, {'evidence': 0.508, 'host': 0.353, 'virus': 0.349, 'infected': 0.319, 'risk': 0.302}, {'clinical': 0.415, 'development': 0.349, 'covid': 0.322, 'studies': 0.314, '19': 0.299}, {'care': 0.558, 'outcomes': 0.435, 'clinical': 0.315, 'management': 0.265, 'data': 0.259}, {'health': 0.552, 'research': 0.342, 'non': 0.323, 'care': 0.319, 'changes': 0.182}, {'public': 0.462, 'disease': 0.308, 'health': 0.275, 'detection': 0.237, 'samples': 0.232}, {'covid': 0.51, '19': 0.474, 'spread': 0.41, 'rate': 0.371, 'virus': 0.264}, {'health': 0.538, 'measures': 0.356, 'public': 0.339, 'outbreak': 0.339, 'research': 0.333}, {'public': 0.496, 'risk': 0.423, 'information': 0.415, 'health': 0.393, 'response': 0.231}]
['evidence', 'host', 'virus', 'infected', 'risk']
['transmission', 'virus', 'health', 'disease', 'care']
['cli

In [77]:
print(task_ner)

[' COVID-19 risk factors epidemiological studies Data potential risks factors Smoking pre-existing pulmonary disease Co-infections co-existing respiratory/viral infections virus virulent co-morbidities Neonates pregnant women Socio-economic behavioral factors economic impact virus Transmission dynamics virus reproductive number incubation period serial interval modes transmission environmental factors Severity of disease risk fatality symptomatic hospitalized patients high-risk patient groups Susceptibility populations Public health mitigation effective control', ' COVID-19 risk factors epidemiological studies Data potential risks factors Smoking pre-existing pulmonary disease Co-infections co-existing respiratory/viral infections virus virulent co-morbidities Neonates pregnant women Socio-economic behavioral factors economic impact virus Transmission dynamics virus reproductive number incubation period serial interval modes transmission environmental factors Severity of disease risk f

In [ ]:
#read in udel topics, create task_only
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3.xml"
write_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-ruir.task.short.xml"
topics_n = 40

tree = Et.parse(topics_path)
root = tree.getroot()

for i in range(topics_n):
    root[i][0].text += " " + list_of_tasks_short[rnd3classes[i]]

#topics = [root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]] for i in range(topics_n)]
#print(topics)

#topics = [t + " " + list_of_tasks_short[rnd3classes[i]] for i,t in enumerate(topics)]
#print(topics)

tree.write(open(write_path, 'wb'))


In [37]:
#read in udel topics, create task_only
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-udel.xml"
write_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-udel-weighted-short.xml"
topics_n = 40

tree = Et.parse(topics_path)
root = tree.getroot()

for i in range(topics_n):
    root[i][0].text += " " + root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]]

#topics = [root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]] for i in range(topics_n)]
#print(topics)

#topics = [t + " " + list_of_tasks_short[rnd3classes[i]] for i,t in enumerate(topics)]
#print(topics)

tree.write(open(write_path, 'wb'))
    
#TODO vergelijk met/zonder narrative
#TODO probeer op nieuwe round data
#TODO vergelijk invloed van tasks over de rondes heen
#TODO fix anserini influence
#manual topic classification based on feedback

#This accuracy might decrease over the rounds as the newer , but that in itself could be worth reporting.


"""
         0.5-4 map    3.5-4 map     0.5-4 p5    3.5-4 p5    0.5-4 p10   3.5-4 p10   0.5-4 bpref    3.5-4 bpref
qq       0.0580       0.0118        0.4550      0.0550      0.3950      0.0550      0.1769         0.1221
qq3      0.0521       0.0115        0.4650      0.0700                              0.1650         0.1176
qdel     0.0782       0.0144        0.6154      0.0462      0.5564      0.0615      0.2014         0.1439
qdel3    0.0737       0.0145        0.5800      0.0550      0.5275      0.0700      0.1980         0.1392
qdel5    0.0672       0.0140        0.5400      0.0800      0.4950      0.0700      0.1896         0.1344
qshort   0.0673       0.0132        0.4800      0.0650      0.4400      0.0550      0.1950         0.1370
"""

"""
tmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j3.5-4.txt test.weighted.short.txt 
map                   	all	0.0132
tmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j3.5-4.txt test.tfidf3.txt 
map                   	all	0.0145
tmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j3.5-4.txt test.tfidf5.txt 
map                   	all	0.0140
tmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j0.5-4.txt test.tfidf3.txt 
map                   	all	0.0737
tmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j0.5-4.txt test.tfidf5.txt 
map                   	all	0.0672
tmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j0.5-4.txt test.weighted.short.txt 
map                   	all	0.0673
tmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j0.5-4.txt test.qdel.txt 
map                   	all	0.0580
"""

'\ntmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j3.5-4.txt test.weighted.short.txt \nmap                   \tall\t0.0132\ntmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j3.5-4.txt test.tfidf3.txt \nmap                   \tall\t0.0145\ntmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j3.5-4.txt test.tfidf5.txt \nmap                   \tall\t0.0140\ntmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j0.5-4.txt test.tfidf3.txt \nmap                   \tall\t0.0737\ntmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j0.5-4.txt test.tfidf5.txt \nmap                   \tall\t0.0672\ntmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j0.5-4.txt test.weighted.short.txt \nmap                   \tall\t0.0673\ntmschoegje@schroefje:~/Desktop/trec_eval/covid$ ./trec_eval -m map qrels-covid_d4_j0.5-4.txt 

In [47]:
limit = 5
print(get_tfidf(task_ner, limit))
tfidf = get_tfidf(task_ner, limit + 5)

#read in basic topics, create ruir.qq.tfidf and ruir.qdel.tfidf
#topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3.xml"
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-udel.xml"
write_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-ruir.qdel.weighted.tfidf" + str(limit) + ".xml"
topics_n = 40

tree = Et.parse(topics_path)
root = tree.getroot()

for i in range(topics_n):
    terms = [term for term in tfidf[rnd3classes[i]]]
    
    numterms = 0
    newterms = []
    for t in terms:
        if t not in stopwords and numterms < limit:
            newterms.append(t)
            numterms += 1
    root[i][0].text += " " + root[i][0].text + " " + " ".join(newterms)
    print(newterms)
    
#    list_of_tasks_short[rnd3classes[i]]

#topics = [root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]] for i in range(topics_n)]
#print(topics)

#topics = [t + " " + list_of_tasks_short[rnd3classes[i]] for i,t in enumerate(topics)]
#print(topics)

tree.write(open(write_path, 'wb'))

[{'transmission': 0.821, 'virus': 0.265, 'health': 0.24, 'disease': 0.202, 'care': 0.185}, {'factors': 0.56, 'risk': 0.37, 'transmission': 0.294, 'virus': 0.285, 'infections': 0.253}, {'evidence': 0.508, 'host': 0.353, 'virus': 0.349, 'infected': 0.319, 'risk': 0.302}, {'clinical': 0.415, 'development': 0.349, 'covid': 0.322, 'studies': 0.314, '19': 0.299}, {'care': 0.558, 'outcomes': 0.435, 'clinical': 0.315, 'management': 0.265, 'data': 0.259}, {'health': 0.552, 'research': 0.342, 'non': 0.323, 'care': 0.319, 'changes': 0.182}, {'public': 0.462, 'disease': 0.308, 'health': 0.275, 'detection': 0.237, 'samples': 0.232}, {'covid': 0.51, '19': 0.474, 'spread': 0.41, 'rate': 0.371, 'virus': 0.264}, {'health': 0.538, 'measures': 0.356, 'public': 0.339, 'outbreak': 0.339, 'research': 0.333}, {'public': 0.496, 'risk': 0.423, 'information': 0.415, 'health': 0.393, 'response': 0.231}]
['evidence', 'host', 'virus', 'infected', 'risk']
['transmission', 'virus', 'health', 'disease', 'care']
['cli

In [50]:
#now qq + tfidf
limit = 5
print(get_tfidf(task_ner, limit))
tfidf = get_tfidf(task_ner, limit + 5)

#read in basic topics, create ruir.qq.tfidf and ruir.qdel.tfidf
topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3.xml"
#topics_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-udel.xml"
write_path = "/home/tmschoegje/Desktop/caos-19/round5/topics.covid-round3-ruir.qq.weighted.tfidf" + str(limit) + ".xml"
topics_n = 40

tree = Et.parse(topics_path)
root = tree.getroot()

for i in range(topics_n):
    terms = [term for term in tfidf[rnd3classes[i]]]
    
    numterms = 0
    newterms = []
    for t in terms:
        if t not in stopwords and numterms < limit:
            newterms.append(t)
            numterms += 1
    root[i][0].text += " " + root[i][0].text + " " + " ".join(newterms)
    print(newterms)
    
#    list_of_tasks_short[rnd3classes[i]]

#topics = [root[i][0].text + " " + list_of_tasks_short[rnd3classes[i]] for i in range(topics_n)]
#print(topics)

#topics = [t + " " + list_of_tasks_short[rnd3classes[i]] for i,t in enumerate(topics)]
#print(topics)

tree.write(open(write_path, 'wb'))

[{'transmission': 0.821, 'virus': 0.265, 'health': 0.24, 'disease': 0.202, 'care': 0.185}, {'factors': 0.56, 'risk': 0.37, 'transmission': 0.294, 'virus': 0.285, 'infections': 0.253}, {'evidence': 0.508, 'host': 0.353, 'virus': 0.349, 'infected': 0.319, 'risk': 0.302}, {'clinical': 0.415, 'development': 0.349, 'covid': 0.322, 'studies': 0.314, '19': 0.299}, {'care': 0.558, 'outcomes': 0.435, 'clinical': 0.315, 'management': 0.265, 'data': 0.259}, {'health': 0.552, 'research': 0.342, 'non': 0.323, 'care': 0.319, 'changes': 0.182}, {'public': 0.462, 'disease': 0.308, 'health': 0.275, 'detection': 0.237, 'samples': 0.232}, {'covid': 0.51, '19': 0.474, 'spread': 0.41, 'rate': 0.371, 'virus': 0.264}, {'health': 0.538, 'measures': 0.356, 'public': 0.339, 'outbreak': 0.339, 'research': 0.333}, {'public': 0.496, 'risk': 0.423, 'information': 0.415, 'health': 0.393, 'response': 0.231}]
['evidence', 'host', 'virus', 'infected', 'risk']
['transmission', 'virus', 'health', 'disease', 'care']
['cli

In [ ]:
#test
#quick query term test
from pyserini.search import SimpleSearcher
#from pyserini.index import pyutils
#from pyserini.analysis import pyanalysis
#from pyserini.search import pysearch
#from pyserini.search import pyquerybuilder
#from pyserini.pyclass import autoclass
#from pyserini.analysis.pyanalysis import get_lucene_analyzer

index_loc = '/home/tmschoegje/Desktop/anserini/indexes/lucene-index-cord19-full-text-2020-05-19/'
searcher = SimpleSearcher(index_loc)
#index_utils = pyutils.IndexReaderUtils(index_loc)


hits = searcher.search('corona origin')

# Print the first 10 hits:
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')
    
    
hits = searcher.search('corona origin origin origin')
print()
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

